# Title

In [5]:
from openai import OpenAI
import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer
import random
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
client = OpenAI()
detokenize = TreebankWordDetokenizer().detokenize

In [375]:
#Importing a list of sentences from a work of literature. Trimming the first/last 10 sentences because intro/ending can be weird.
corpus = nltk.corpus.gutenberg.sents('austen-emma.txt')[10:-10] + nltk.corpus.gutenberg.sents('austen-persuasion.txt')[10:-10] + nltk.corpus.gutenberg.sents('austen-sense.txt')[10:-10]

#I need to get rid of quotation marks, especially at the end; they break everything
corpus = [[w for w in s if w not in  '."?.!'] for s in sentences]

#Try to limit the cost by considering sentences of between 5 and 10 words.
corpus = [s for s in sentences if 4< len(s) < 11]

### Our corpus is the collection of sentences from Jane Austen's three works in the gutenberg corpora. 

In [340]:
'Out filtered corpus contains {} sentences'.format(len(corpus))

'Out filtered corpus contains 545 sentences'

Some useful functions for categorizing sentences, quickly getting a handle on those sentences.

In [341]:
def tags_present_in_sentence(s):
    tagged = nltk.pos_tag(s)
    tags = [w[1] for w in tagged]
    return tags

def display_sampling(corpus, condition, k = 5, seed = None):
    random.seed(seed)
    to_display = random.choices([s for s in corpus if condition(s)], k = k)
    for s in to_display:
        print(detokenize(s))
        
def num_satisfying(corpus, condition):
    return len([s for s in corpus if condition(s)]) 

In [342]:
#conditions

def ends_in_VBD(s):
    "Ends in a verb in the past tense"
    return nltk.pos_tag(s)[-1][1] in ['VBD']

def ends_in_married(s):
    return s[-1] == 'married'

def contains_fairfax(s):
    return 'Fairfax' in s

def contains_two(s):
    return 'two' in s or 'Two' in s

def contains_CD(s):
    "Contains a numerical reference"
    return "CD" in tags_present_in_sentence(s)

In [343]:
conditions = [ends_in_VBD, ends_in_married, contains_fairfax, contains_two, contains_CD]

In [357]:
num_satisfying(corpus, ends_in_VBD)

14

In [358]:
display_sampling(corpus, ends_in_VBD)

Just as they were setting off, the gentlemen returned
I am not like Jane; I wish I were
He seemed to me very well off as he was
His disapprobation was expressed, but apparently very little regarded
cried Mary in an ecstasy, just as I said


The above is helpful for playing around and getting sense of what our sentences look like. Now build a dataframe including the sentences and the result of our conditions.

In [348]:
def add_condition_column(df, cond):
    df[cond.__name__] = df.sentences.apply(cond)


In [349]:
def df_from_corpus_and_conditions(corpus, conditions):
    df = pd.DataFrame(pd.Series(corpus, name = 'sentences'))
    for cond in conditions:
        add_condition_column(df, cond)
    return df
    

In [359]:
df = df_from_corpus_and_conditions(corpus, conditions)

In [360]:
df.head()

,sentences,ends_in_VBD,ends_in_married,contains_fairfax,contains_two,contains_CD
0,"[The, event, had, every, promise, of, happines...",False,False,False,False,False
1,"[Poor, Miss, Taylor, !--, I, wish, she, were, ...",False,False,False,False,False
2,"[My, dear, ,, how, am, I, to, get, so, far]",False,False,False,False,False
3,"[We, must, go, in, the, carriage, ,, to, be, s...",False,False,False,False,False
4,"[We, talked, it, all, over, with, Mr, Weston, ...",False,False,False,False,False


Now that we have a dataset (and a suite of tools for adding to the dataset), let's focus on getting the LLM to learn the condition in context and evaluate the LLM's learning. 

In [361]:
def split_train_test(df, cond, train_k = 5, test_k = 25, seed = None):

    satisfying = df.loc[df[cond.__name__]== True]
    try: train, test = train_test_split(satisfying, train_size = train_k, test_size = test_k, random_state = seed) #might not be enough samples to use test_k
    except: train, test = train_test_split(satisfying, train_size = train_k, random_state = seed) #in which case, just use the remainder

    not_satisfying = df.loc[df[cond.__name__]== False]
    train = pd.concat([train, not_satisfying.sample(train.shape[0], random_state = seed)])
    test = pd.concat([test, not_satisfying.sample(test.shape[0], random_state = seed)])
    test = test.sample(frac = 1) #this randomizes the order, so that we don't show a bunch of True followed by a bunch of False


    return train, test

In [363]:
train, test = split_train_test(df, ends_in_VBD)

In [365]:
def prompt_from_train_test_cond(train, test, cond):  
    p = "The following sentences are labeled 'True' if the rule is satisfied and labeled 'False' otherwise:\n\n"
    
    for s, label in zip(train['sentences'], train[cond.__name__]):
        p += detokenize(s) + ": " + str(label)+'\n'

    p += "\nThink about what the sentences labeled true all have in common, and label the following sentences according to the rule. Return an ordered list using only the words 'True' or 'False':\n\n"

    for s in test['sentences']:
        p += detokenize(s) + '\n'
    
    return p

In [366]:
print(prompt_from_train_test_cond(train, test, ends_in_VBD))

The following sentences are labeled 'True' if the rule is satisfied and labeled 'False' otherwise:

I am not like Jane; I wish I were: True
I hope you do not retract what you then said: True
It was every day implied, but never professedly declared: True
By his style, I should imagine it just settled: True
The weather was remarkably fine, and she readily consented: True
I shall be happier to burn it," replied Harriet: False
I will tell you why she is out of spirits: False
You are very kind to bring me these interesting particulars: False
I must endeavour to subdue my mind to my fortune: False
Lady Middleton proposed a rubber of Casino to the others: False

Think about what the sentences labeled true all have in common, and label the following sentences according to the rule. Return an ordered list using only the words 'True' or 'False':

Say ` No,' if it is to be said
They would catch worse colds at the Crown than anywhere
Just as they were setting off, the gentlemen returned
I have bee

### Putting it all together now:

In [221]:
df = df_from_corpus_and_conditions(corpus, conditions)    

In [289]:
df.head(1)

,sentences,ends_in_VBD,ends_in_married,contains_fairfax,contains_two,contains_CD
0,"[It, was, Miss, Taylor, ', s, loss, which, fir...",False,False,False,False,False


In [335]:
train, test = split_train_test(df, ends_in_married)

In [367]:
p = prompt_from_train_test_cond(train, test, ends_in_married)

In [368]:
print(p)

The following sentences are labeled 'True' if the rule is satisfied and labeled 'False' otherwise:

I am not like Jane; I wish I were: False
I hope you do not retract what you then said: False
It was every day implied, but never professedly declared: False
By his style, I should imagine it just settled: False
The weather was remarkably fine, and she readily consented: False
I shall be happier to burn it," replied Harriet: False
I will tell you why she is out of spirits: False
You are very kind to bring me these interesting particulars: False
I must endeavour to subdue my mind to my fortune: False
Lady Middleton proposed a rubber of Casino to the others: False

Think about what the sentences labeled true all have in common, and label the following sentences according to the rule. Return an ordered list using only the words 'True' or 'False':

Say ` No,' if it is to be said
They would catch worse colds at the Crown than anywhere
Just as they were setting off, the gentlemen returned
I hav

In [369]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "I have a secret rule in mind. Your only goal is to learn that rule."},
    {"role": "user", "content": p}
  ]
)


In [370]:
def get_bool_response(response):
    r_str = response.choices[0].message.content.split(',')
    r = ['True' in label for label in r_str]
    return r

In [371]:
r = get_bool_response(response)

In [372]:
r

[False,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False]

In [373]:
accuracy = [x == y for x, y in zip(list(test.ends_in_married), r)]

In [374]:
correct = 0
for x in accuracy:
    if x: correct +=1
print(correct/len(accuracy))

0.6428571428571429
